# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from itertools import islice
from pprint import pprint


# Import API key
from config2 import gkey2


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
allcitiesdf = pd.read_csv("weatherdatabycity.csv")
allcitiesdf.head()

,Unnamed: 0,CityName,Latitude,Longitude,Temperature,Clouds,Humidity,Windspeed,pw
0,0,Zyryanka,65.75,150.85,-28.52,100,96,4.21,26.283731
1,1,Hobart,-42.88,147.33,65.84,86,64,1.01,8.326025
2,2,Bredasdorp,-34.53,20.04,66.13,31,68,21.92,9.045226
3,3,Vaini,-21.20,-175.20,78.55,25,87,19.64,24.150875
4,4,Jamestown,42.10,-79.24,35.06,100,93,4.00,-5.857438


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey2)
locations = allcitiesdf[["Latitude", "Longitude"]].astype(float)
humidity = allcitiesdf["Humidity"].astype(float)

#create a the base figure
fig = gmaps.figure()
#add a heatmap layer using humidity data as the weighting scheme
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#narrow down cities to only those I'd want to vacation in
vacationdf = allcitiesdf.loc[(allcitiesdf["Temperature"] > 75)
                            & (allcitiesdf["Temperature"] < 85)
                            & (allcitiesdf["Humidity"] > 20)
                            & (allcitiesdf["Humidity"] < 70)
                            & (allcitiesdf["Windspeed"] < 18)
                            & (allcitiesdf["Clouds"] <10)]
print(f'We have narrowed down our search to {len(vacationdf)} cities with excellent weather!')

vacationdf.reset_index(inplace=True)
vacationdf



We have narrowed down our search to 14 cities with excellent weather!


,index,Unnamed: 0,CityName,Latitude,Longitude,Temperature,Clouds,Humidity,Windspeed,pw
0,27,27,Wahiawa,21.50,-158.02,78.37,6,57,5.01,15.688155
1,71,71,Barra de Tecoanapa,16.52,-98.75,80.96,0,64,6.87,19.842091
2,86,86,Kahului,20.89,-156.47,77.72,7,69,14.00,18.406091
3,165,165,Las Compuertas,25.45,-108.35,76.32,0,65,7.00,16.175326
4,181,181,Lahaina,20.88,-156.68,77.70,7,69,14.00,18.388210
5,207,207,Upington,-28.45,21.26,77.00,0,41,17.22,10.556978
6,228,228,Santiago,-33.46,-70.65,79.99,0,30,4.09,8.904183
7,247,247,Santiago del Estero,-27.80,-64.26,84.31,0,33,7.45,11.797056
8,382,382,Tsabong,-26.02,22.40,77.41,6,47,8.63,12.349208
9,390,390,Broken Hill,-31.95,141.43,80.67,0,30,13.38,9.181625


In [5]:
gmaps.configure(api_key=gkey2)
locations = vacationdf[["Latitude", "Longitude"]].astype(float)
humidity = vacationdf["Humidity"].astype(float)

#create a the base figure
fig = gmaps.figure()
#add a heatmap layer using humidity data as the weighting scheme
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(columns={"Hotel Name","City Name","Country","Rating","Latitude","Longitude"})
hotel_df


,Latitude,Country,Longitude,Hotel Name,City Name,Rating


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set a limiter for api calls to prevent excess usage during development
#Currently set not to limit -- will run for all rows in vacationdf
apicalllimit = len(vacationdf["CityName"])

# use iterrows to iterate through pandas dataframe
for index, row in islice(vacationdf.iterrows(), apicalllimit):
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    coords = f'{vacationdf["Latitude"][index]},{vacationdf["Longitude"][index]}',
    cname = vacationdf["CityName"][index]
    params = {
                "location": coords,
                "keyword": "hotel",
                "radius": 5000,
                "type": "hotel",
                "key": gkey2,
            }

    #Tell the user which city they're searching for
    #Make API call for that city
    print(f'Searching for Hotels in: {vacationdf["CityName"][index]}.')
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Separate API request (this time to geocode) to get country name since that's not in the first API data set
    base_url_2 = ('https://maps.googleapis.com/maps/api/geocode/json?'
        'address={0}&key={1}').format(cname, gkey2)
    data = requests.get(base_url_2).json() 
    
    #put our data in the hotel_df, with some error handling
    try:
        print(f"We found a hotel in {cname} called {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'City Name'] = cname
        hotel_df.loc[index, 'Country'] = data["results"][0]["address_components"][3]["short_name"]
        hotel_df.loc[index, 'Rating'] = results[0]['rating']
        hotel_df.loc[index, 'Latitude'] = data["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, 'Longitude'] = data["results"][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Something went wrong. We were unable to find a hotel.")


Searching for Hotels in: Wahiawa.
We found a hotel in Wahiawa called The Inn At Schofield.
Searching for Hotels in: Barra de Tecoanapa.
We found a hotel in Barra de Tecoanapa called Hotel Garapacho.
Something went wrong. We were unable to find a hotel.
Searching for Hotels in: Kahului.
We found a hotel in Kahului called Courtyard by Marriott Maui Kahului Airport.
Searching for Hotels in: Las Compuertas.
Something went wrong. We were unable to find a hotel.
Searching for Hotels in: Lahaina.
We found a hotel in Lahaina called The Westin Maui Resort & Spa, Ka'anapali.
Searching for Hotels in: Upington.
We found a hotel in Upington called The Maxton Boutique Hotel Upington.
Searching for Hotels in: Santiago.
We found a hotel in Santiago called Four Points by Sheraton Santiago.
Searching for Hotels in: Santiago del Estero.
We found a hotel in Santiago del Estero called Hotel Solano.
Searching for Hotels in: Tsabong.
We found a hotel in Tsabong called Dikukama Hotel.
Searching for Hotels in:

In [18]:
hotel_df.dropna(inplace=True)
hotel_df

,Latitude,Country,Longitude,Hotel Name,City Name,Rating
0,21.501,US,-158.021,The Inn At Schofield,Wahiawa,3.2
2,20.8893,US,-156.473,Courtyard by Marriott Maui Kahului Airport,Kahului,4.3
4,20.8783,US,-156.683,"The Westin Maui Resort & Spa, Ka'anapali",Lahaina,4.3
5,-28.3953,ZA,21.2368,The Maxton Boutique Hotel Upington,Upington,4.6
6,-33.4489,CL,-70.6693,Four Points by Sheraton Santiago,Santiago,4.4
7,-27.7834,AR,-64.2642,Hotel Solano,Santiago del Estero,4.1
8,-26.0208,BW,22.4113,Dikukama Hotel,Tsabong,3.7
9,-31.9539,2880,141.454,ibis Styles Broken Hill,Broken Hill,4.1
10,-32.9533,Mendoza,-68.7881,Hotel Ychoalay Caz,Reconquista,3.7
11,-26.8083,AR,-65.2176,Hilton Garden Inn Tucuman,San Miguel de Tucuman,4.4


In [19]:
# NOTE: Do not change any of the code in this cell
#narrowed_city_df
locations = hotel_df[["Latitude", "Longitude"]]

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)



In [21]:
# Add marker layer ontop of heat map
fig.add_layer(hotel_info)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))